# snowflake

> Working with Snowflake

In [ ]:
#| default_exp snowflake

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from triggerkit import util
from triggerkit import actions
import snowflake.connector
import os, re
import pandas as pd
from snowflake.connector.connection import SnowflakeConnection
from snowflake.connector.cursor import SnowflakeCursor
from typing import Dict, List, Callable, Optional, Any, Union

In [ ]:
#| export

util.VIEW_REGISTRY: Dict[str, Dict[str, Any]] = {}

def register_view(name: str, query: str, metadata: Optional[Dict[str, Any]] = None):
    """
    Register a Snowflake view with the system.
    
    Args:
        name: Name of the view
        query: SQL query or view name to fetch datay
        metadata: Additional metadata about the view
        
    Returns:
        None
    """
    util.VIEW_REGISTRY[name] = {
        'query': query,
        'metadata': metadata or {}
    }
    util.logger.info(f"Registered view: {name}")

In [ ]:
#| export

@actions.register('Register Views','Registers all views in data with VIEW_REGISTRY.')
def register_views(data: List[Dict[str, Any]], context = None):
    """
    Registers all views in data with VIEW_REGISTRY.
    """
    for view in data:
        register_view(
            view.get('name'),
            view.get('query'),
            {'description': view.get('description')}
            )

2025-05-23 00:17:09,729 - triggerkit - INFO - Registered action: Register Views
2025-05-23 00:17:09,731 - triggerkit - WARNING - Action 'Register Views' already exists. Keeping the existing one.


In [ ]:
#| export 

util.snowflake_connection = None

def connect(sf_config: Dict[str, Any]) -> SnowflakeConnection:
    """
    Establish connection to Snowflake.
    
    Args:
        sf_config: Dictionary containing Snowflake connection parameters
        
    Returns:
        Snowflake connection object
    """
    # Check for environment variables for sensitive data
    password = os.environ.get('SNOWFLAKE_PASSWORD') or sf_config.get('password')
    
    try:
        connection = snowflake.connector.connect(
            user=sf_config.get('user'),
            password=password,
            account=sf_config.get('account'),
            warehouse=sf_config.get('warehouse'),
            database=sf_config.get('database'),
            schema=sf_config.get('schema')
        )
        util.logger.info("Successfully connected to Snowflake")
        return connection
    except Exception as e:
        util.logger.error(f"Failed to connect to Snowflake: {str(e)}")
        raise

In [ ]:
#| export

def query(sql_query: str) -> List[Dict[str, Any]]:
    """
    Execute a query and return results.
    
    Args:
        sql_query: SQL query to execute
        
    Returns:
        List of result rows as dictionaries
    """
    if not hasattr(util, 'snowflake_connection') or  hasattr(util, 'snowflake_connection') and util.snowflake_connection == None:
        util.snowflake_connection = connect(util.SNOWFLAKE_CONFIG)
    try:
        cursor: SnowflakeCursor = util.snowflake_connection.cursor()
        cursor.execute(sql_query)
        column_names = [col[0] for col in cursor.description]
        
        # Convert results to list of dictionaries
        results = []
        for row in cursor:
            results.append(dict(zip(column_names, row)))
        cursor.close()
        return results
    except Exception as e:
        print('Query:', sql_query)
        util.logger.error(f"Failed to execute query: {str(e)}")
        raise Exception(e)

def get_view_data(view_name: str) -> List[Dict[str, Any]]:
    """
    Execute a query for a registered view and return results.
    
    Args:
        view_name: Name of the registered view
        
    Returns:
        List of result rows as dictionaries
    """
    if view_name not in util.VIEW_REGISTRY:
        raise ValueError(f"View '{view_name}' is not registered")
    
    view_config = util.VIEW_REGISTRY[view_name]
    sql_query = view_config['query']
    
    try:
        results = query(sql_query)
        util.logger.info(f"Successfully executed query for view '{view_name}', returned {len(results)} rows")
        return results
    except Exception as e:
        util.logger.error(f"Failed to execute query for view '{view_name}': {str(e)}")
        raise

In [ ]:
#| export

def extract_view_sql(ddl: str) -> str:
    """
    Extracts the SQL query from a Snowflake view definition.
    
    Args:
        ddl: Snowflake view definition
    
    Returns:
        Extracted SQL query
    """
    if 'CREATE' not in ddl.upper():
        util.logger.warning('No CREATE statement found in DDL')
        return ddl
    
    ddl = ddl.strip().replace(';', '')
    
    # Find the AS keyword (case-insensitive)
    match = re.search(r'\bAS\b\s*', ddl, flags=re.IGNORECASE)
    if not match:
        util.logger.warning('No AS statement found in DDL')
        return ddl
    
    # Extract everything after AS
    start = match.end()
    sql = ddl[start:].strip()

    # Check if the SQL starts with a parenthesis
    if sql.startswith('('):
        # Find the matching closing parenthesis
        balance = 1  # Start with 1 for the opening parenthesis
        for i, c in enumerate(sql[1:], 1):
            if c == '(':
                balance += 1
            elif c == ')':
                balance -= 1
                if balance == 0:  # Found the matching closing parenthesis
                    return sql[1:i].strip()
    
    return sql


In [ ]:
test_ddl1 = """
create or replace view EXAMPLE_VIEW_1(
	TITLE,
	TITLE_LINK,
	TEXT,
	OPTION_NAME,
	OPTION_VALUE,
	CONFIG
) as 
    select *
    from main.tk_testing.trigger_kit_example_base;"""

extract_view_sql(test_ddl1)

'select *\n    from main.tk_testing.trigger_kit_example_base'

In [ ]:
#| export

@actions.register('Get DDL and Register Views','Given view names, it will retrieve the views\' definition and register the views.')
def get_ddl_and_register(data, context = None):
    """
    For each view in data, retrieve the DDL and register the view. 
    """
    database = util.SNOWFLAKE_CONFIG.get('database')
    check_schema = util.SNOWFLAKE_CONFIG.get('check_schema')

    results = []

    for view_info in data:
        view_name = view_info['TABLE_NAME']

        ddl_query = f"""
            SELECT GET_DDL('VIEW', '{database}.{check_schema}.{view_name}') AS full_ddl
        """
        view_ddl_result = query(ddl_query)

        if view_ddl_result:
            raw_ddl = view_ddl_result[0]['FULL_DDL']
            cleaned_sql = extract_view_sql(raw_ddl)

            results.append({
                'name': view_name,
                'query': cleaned_sql
            })

    register_views(results)

2025-05-23 00:17:09,755 - triggerkit - INFO - Registered action: Get DDL and Register Views
2025-05-23 00:17:09,756 - triggerkit - WARNING - Action 'Get DDL and Register Views' already exists. Keeping the existing one.


In [ ]:
#| exporti

def _check_schema(config: Dict[str, Any]):
    """
    Retrieves all views in 'check_schema' and extracts only the SELECT/WITH logic from the view definition.
    """
    if not config.get('snowflake') or not config['snowflake'].get('check_schema'):
        return None
    print('Running _check_schema function')

    database = config['snowflake'].get('database')
    check_schema = config['snowflake'].get('check_schema')

    # Query to get all of the views defined in the check_schema
    views_query = f"""
        SELECT table_name
        FROM {database}.INFORMATION_SCHEMA.VIEWS
        WHERE table_schema = '{check_schema}'
        """
    
    # Query to get all of the views that contain TK_CONFIG or CONFIG columns
    views_with_config_cols_query = f"""
        SELECT 
            distinct table_name
        FROM {database}.INFORMATION_SCHEMA.COLUMNS
        WHERE table_schema = '{check_schema}'
        AND column_name ILIKE ANY ('TK_CONFIG', 'CONFIG')
        AND data_type ILIKE ANY ('VARIANT', 'OBJECT')
        AND table_name in (
                SELECT distinct table_name
                FROM {database}.INFORMATION_SCHEMA.VIEWS
                WHERE table_schema = '{check_schema}')
        """

    try:
        view_names_list = query(views_query)
        print('view_names_list:', view_names_list)
        print('views_query:', views_query)
        print('views_with_config_cols_query:', views_with_config_cols_query)
        return view_names_list, views_query, views_with_config_cols_query

    except Exception as e:
        print(f"Error executing query: {e}")
        return None

In [ ]:
#| export

def _check_schema(config: Dict[str, Any]):
    """
    Retrieves all views in 'check_schema' and constructs queries for view registration.
    """
    util.logger.debug("Entering _check_schema function.")
    if not config.get('snowflake'):
        util.logger.warning("Snowflake configuration missing in 'config'. Cannot check schema.")
        return None, None, None
        
    snowflake_cfg = config['snowflake']
    if not snowflake_cfg.get('check_schema'):
        util.logger.info("'check_schema' not defined in Snowflake configuration. Skipping schema check for dynamic views.")
        return None, None, None

    database = snowflake_cfg.get('database')
    check_schema_name = snowflake_cfg.get('check_schema') # Renamed to avoid confusion with function name

    if not database or not check_schema_name:
        util.logger.error(f"Database ('{database}') or check_schema ('{check_schema_name}') not fully configured in [snowflake] section. Cannot proceed with _check_schema.")
        return None, None, None

    util.logger.info(f"Checking schema '{database}.{check_schema_name}' for views.")

    # Query to get all of the views defined in the check_schema
    views_query_template = f"SELECT table_name FROM {database}.INFORMATION_SCHEMA.VIEWS WHERE table_schema = '{check_schema_name}'"
    
    # Query to get all of the views that contain TK_CONFIG or CONFIG columns
    views_with_config_cols_query_template = f"""
        SELECT DISTINCT table_name
        FROM {database}.INFORMATION_SCHEMA.COLUMNS
        WHERE table_schema = '{check_schema_name}'
        AND column_name ILIKE ANY ('TK_CONFIG', 'CONFIG', 'VIEW_CONFIG') 
        AND data_type ILIKE ANY ('VARIANT', 'OBJECT')
        AND table_name IN (
            SELECT DISTINCT table_name
            FROM {database}.INFORMATION_SCHEMA.VIEWS
            WHERE table_schema = '{check_schema_name}'
        )
        """
    try:
        util.logger.debug(f"Executing views_query: {views_query_template}")
        view_names_list = query(views_query_template) # This returns list of dicts [{'TABLE_NAME': 'name1'}, ...]
        
        if view_names_list is None: # query() could theoretically return None on some error, though it usually raises
             util.logger.warning(f"Query for view names in schema '{check_schema_name}' returned None.")
             return None, views_query_template, views_with_config_cols_query_template


        util.logger.info(f"Found {len(view_names_list)} views in schema '{check_schema_name}'.")
        util.logger.debug(f"View names list from schema '{check_schema_name}': {view_names_list}")
        
        # view_names_list is what get_ddl_and_register expects (list of dicts with 'TABLE_NAME')
        return view_names_list, views_query_template, views_with_config_cols_query_template

    except Exception as e:
        util.logger.error(f"Error executing query in _check_schema for schema '{check_schema_name}': {str(e)}", exc_info=True)
        # Return the query templates even on error, so init_view_registry can still try to register them if desired
        return None, views_query_template, views_with_config_cols_query_template

In [ ]:
#| export

def get_view_columns(view_name: str) -> List[str]:
    """
    Get the columns of a view.
    
    Args:
        view_name: Name of the view
        
    Returns:
        List of column names
    """
    if not hasattr(util, 'snowflake_connection') or  hasattr(util, 'snowflake_connection') and util.snowflake_connection == None:
        util.snowflake_connection = connect(util.SNOWFLAKE_CONFIG)
    
    cursor: SnowflakeCursor = util.snowflake_connection.cursor()
    
    try:
        cursor.execute(f"DESCRIBE VIEW {view_name}")
        columns = [col[0] for col in cursor.fetchall()]
        return columns
    except Exception as e:
        util.logger.error(f"Failed to get columns for view '{view_name}': {str(e)}")
        raise

In [ ]:
#| export

def init_view_registry(config: Dict[str, Any]):
    """
    Starts View Registry by including all locally defined views and including all views
    located in the the schema defined in the configuration file.
    
    Args:
        config: Dictionary containing view configurations
    
    Returns:
        None
    """

    if config.get('views'): register_views(config.get('views'))
    
    schema_views, check_schema_query, views_with_config_cols_query = _check_schema(config)

    if schema_views: 
        print('Registering views from schema:', schema_views)
        get_ddl_and_register(schema_views)
        register_view('tk_check_schema', check_schema_query)
        register_view('tk_configured_views', views_with_config_cols_query)

In [ ]:
#| export

def init_view_registry(config: Dict[str, Any]):
    """
    Starts View Registry by including all locally defined views and including all views
    located in the the schema defined in the configuration file.
    
    Args:
        config: Dictionary containing view configurations
    
    Returns:
        None
    """
    util.logger.info("Initializing View Registry...")

    # 1. Register views from the main configuration file (e.g., TOML [[views]])
    config_defined_views = config.get('views', None)
    if config_defined_views:
        util.logger.debug(f"Found {len(config_defined_views)} views defined in configuration file. Attempting to register them.")
        try:
            register_views(config_defined_views) # plural register_views
        except Exception as e:
            util.logger.error(f"Error registering views from configuration file: {str(e)}", exc_info=True)
            # Decide if this is fatal. For now, we'll continue to try schema views.
    else:
        util.logger.debug("No views found directly in configuration file under 'views' key.")

    # 2. Discover and register views from the specified Snowflake schema
    util.logger.debug("Attempting to discover views from Snowflake schema specified in 'check_schema'.")
    schema_views_data, check_schema_query_str, views_with_config_cols_query_str = _check_schema(config) #

    # schema_views_data is a list of dicts like [{'TABLE_NAME': 'view1'}, ...] or None
    if schema_views_data: 
        util.logger.debug(f"Successfully retrieved {len(schema_views_data)} view names from 'check_schema'. Proceeding with DDL processing and registration.")
        try:
            get_ddl_and_register(schema_views_data) # This will register the views found in the schema
        except Exception as e:
            util.logger.error(f"Error during DDL processing and registration for schema views: {str(e)}", exc_info=True)
    elif schema_views_data is None and config.get('snowflake', {}).get('check_schema'):
        util.logger.warning(f"'check_schema' is configured, but no views were found or an error occurred in _check_schema. DDL processing skipped.")
    else: # No check_schema configured or explicitly no views.
        util.logger.debug("No views to process from 'check_schema' (either not configured, no views found, or error during retrieval).")


    # 3. Register the internal 'tk_check_schema' and 'tk_configured_views'
    # These are registered using their generated SQL query strings.
    # It's important these queries are valid even if _check_schema had issues fetching view_names_list
    # as long as check_schema_query_str and views_with_config_cols_query_str were generated.
    if check_schema_query_str:
        util.logger.debug("Registering internal view: tk_check_schema")
        register_view('tk_check_schema', check_schema_query_str, {'description': 'Internal view for checking schema views.'}) #
    else:
        util.logger.warning("Could not register 'tk_check_schema' because its query string was not generated (likely 'check_schema' not configured).")

    if views_with_config_cols_query_str:
        util.logger.debug("Registering internal view: tk_configured_views") # Corrected typo from 'tk_configured_views' to 'tk_configured_views' if intended
        register_view('tk_configured_views', views_with_config_cols_query_str, {'description': 'Internal view for configured views in schema.'}) #
    else:
        util.logger.warning("Could not register 'tk_configured_views' because its query string was not generated (likely 'check_schema' not configured).")
    
    util.logger.debug(f"View Registry initialization complete. Total views registered: {len(util.VIEW_REGISTRY)}")
    util.logger.debug(f"Current VIEW_REGISTRY keys: {list(util.VIEW_REGISTRY.keys())}")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()